In [47]:
import json

In [48]:
import pandas as pd

In [49]:
from sentence_transformers import SentenceTransformer

In [50]:
from elasticsearch import Elasticsearch

In [51]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [52]:
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer, models

# Load the model and tokenizer manually
model = AutoModel.from_pretrained("/home/mahtab/transformers/all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("/home/mahtab/transformers/all-MiniLM-L6-v2")

# Create a SentenceTransformer model with mean pooling
word_embedding_model = models.Transformer(model_name_or_path="/home/mahtab/transformers/all-MiniLM-L6-v2")
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
sentence_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [53]:
for doc in documents:
    question = doc['question']
    text = doc['text']
    qt = question+' '+text
    
    doc['question_vector'] = sentence_model.encode(question)
    doc['text_vector'] = sentence_model.encode(text)
    doc['question_text_vector'] = sentence_model.encode(qt)

In [54]:
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

es_client = Elasticsearch('https://localhost:9200',
                           http_auth=('elastic', 'xMR77YWEPbFGsMJvin_7'),
                           verify_certs=False) 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

/home/mahtab/.local/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/tmp/ipykernel_25970/3782321256.py:5: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es_client = Elasticsearch('https://localhost:9200',


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [55]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [56]:
course = "data-engineering-zoomcamp"
query = 'I just discovered the course. Can I still join it?'


In [57]:
v_q = sentence_model.encode(query)


In [58]:
knn_query = {
    "field": "text_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000,
    "boost": 0.5,
    "filter": {
        "term": {
            "course": course
        }
    }
}

In [59]:
keyword_query = {
    "bool": {
        "must": {
            "multi_match": {
                "query": query,
                "fields": ["question^3", "text", "section"],
                "type": "best_fields",
                "boost": 0.5,
            }
        },
        "filter": {
            "term": {
                "course": course
            }
        }
    }
}

In [60]:
response = es_client.search(
    index=index_name,
    query=keyword_query,
    knn=knn_query,
    size=5
)

# Hybrid search pipeline


In [61]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')


In [62]:
ground_truth = df_ground_truth.to_dict(orient='records')


In [63]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [64]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [65]:
def elastic_search_hybrid(field, query, vector, course):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text", "section"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }

    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 5,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [66]:
def question_hybrid(q):
    question = q['question']
    course = q['course']

    v_q = sentence_model.encode(question)

    return elastic_search_hybrid('question_vector', question, v_q, course)

In [67]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in ground_truth:
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [68]:
evaluate(ground_truth, question_hybrid)


{'hit_rate': 0.9231934227607097, 'mrr': 0.8484710803404011}

In [69]:
def text_hybrid(q):
    question = q['question']
    course = q['course']

    v_q = sentence_model.encode(question)

    return elastic_search_hybrid('text_vector', question, v_q, course)

In [70]:
evaluate(ground_truth, text_hybrid)

{'hit_rate': 0.9218952834270878, 'mrr': 0.8463183326121448}

In [71]:
def question_text_hybrid(q):
    question = q['question']
    course = q['course']

    v_q = sentence_model.encode(question)

    return elastic_search_hybrid('question_text_vector', question, v_q, course)



In [72]:
evaluate(ground_truth, question_text_hybrid)

{'hit_rate': 0.9255733448723497, 'mrr': 0.851528919659599}

In [ ]:
def elastic_search_hybrid_rrf(field, query, vector, course):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text", "section"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }

    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 5,
        "rank": {
            "rrf": {}
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
course = "data-engineering-zoomcamp"


In [ ]:
query = 'I just discovered the course. Can I still join it?'


In [ ]:
v_q = sentence_model.encode(query)


In [ ]:
elastic_search_hybrid_rrf('question_text_vector', query, v_q, course)


In [73]:
def compute_rrf(rank, k=60):
    """ Our own implementation of the relevance score """
    return 1 / (k + rank)

def elastic_search_hybrid_rrf(field, query, vector, course, k=60):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 10,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text", "section"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }

    knn_results = es_client.search(
        index=index_name, 
        body={
            "knn": knn_query, 
            "size": 10
        }
    )['hits']['hits']
    
    keyword_results = es_client.search(
        index=index_name, 
        body={
            "query": keyword_query, 
            "size": 10
        }
    )['hits']['hits']
    
    rrf_scores = {}
    # Calculate RRF using vector search results
    for rank, hit in enumerate(knn_results):
        doc_id = hit['_id']
        rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Adding keyword search result scores
    for rank, hit in enumerate(keyword_results):
        doc_id = hit['_id']
        if doc_id in rrf_scores:
            rrf_scores[doc_id] += compute_rrf(rank + 1, k)
        else:
            rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Sort RRF scores in descending order
    reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get top-K documents by the score
    final_results = []
    for doc_id, score in reranked_docs[:5]:
        doc = es_client.get(index=index_name, id=doc_id)
        final_results.append(doc['_source'])
    
    return final_results

In [74]:
def question_text_hybrid_rrf(q):
    question = q['question']
    course = q['course']

    v_q = sentence_model.encode(question)

    return elastic_search_hybrid_rrf('question_text_vector', question, v_q, course)

evaluate(ground_truth, question_text_hybrid_rrf)

{'hit_rate': 0.9545651233232367, 'mrr': 0.8754543487667681}